### Task 158: V2 liquidity fix
* Ti: Token input
* To: Token output
* Tr: Token ratio
* R: Reserve (total)
* F: Fee imposed on swap (ie, 0.3%)
* Ri: Reserve Input
* Ro: Reserve Output
* Ai: Amount of input token 
* Ao: Amount of output token 
* S: actual swap amount in execution in output token

### References
* https://docs.uniswap.org/protocol/V2/concepts/protocol-overview/how-uniswap-works
* https://ethereum.org/en/developers/tutorials/uniswap-v2-annotated-code/
* https://uniswap.org/whitepaper.pdf
* https://medium.com/@chiqing/uniswap-v2-explained-beginner-friendly-b5d2cb64fe0f
* https://jeiwan.net/posts/programming-defi-uniswapv2-4/
* https://betterprogramming.pub/uniswap-v2-in-depth-98075c826254
* https://docs.uniswap.org/protocol/V2/concepts/core-concepts/swaps

### Solidity Code
* https://github.com/Uniswap/v2-core/blob/master/contracts/UniswapV2Pair.sol#L159
* https://github.com/Uniswap/v2-periphery/blob/master/contracts/libraries/UniswapV2Library.sol

In [1]:
from IPython.display import Image
import os
cwd =  os.getcwd().replace("notebooks/research/ian","")
os.chdir(cwd)

#### Mathematical Mapping Legend
* reserveIn (Ri) -> x
* reserveOut (Ro) -> y
* amountIn (Ai) -> $\Delta$x
* amountOut (Ao) -> $\Delta$y
* 1 - swap fee -> r = (1 - 0.003) or (997/1000)

In [2]:
Ti = 16 # Token input
To = 13 # Token output
R = 1000 # Total reserve
F = 997 # Fee imposed on swap
Ao = 10 # Amount of output token you want

How much LP to remove

$\Delta$k = xy - (x - r$\Delta$x)(y - $\Delta$y)

* reserve0 (Ri) -> x
* reserve1 (Ro) -> y
* amountOut (Ao) -> $\Delta$y
* k -> k

$\Delta$ x = x $\left(1-\sqrt{1 - \frac{\Delta k}{k}} \right)$

$\Delta$ y = y $\left(1-\sqrt{1 - \frac{\Delta k}{k}} \right)$

(x - $\Delta$x)(y - $\Delta$y) = k - $\Delta$ k

### Extract LP from one of the token reserves
Start with this
> (x)(y - r$\Delta$y) = k - $\Delta$ k

Do the algebra, and we have amount LP out:
> $\Delta$ k = xr$\Delta$y

Likewise, we have amount desired token required:
> $\Delta$ y = $\frac{r \Delta k}{x}$

In [3]:
def getAmountLPOut(Ai,x):
    return x*997*Ai/1000

def getAmountIn4LP(Ao,x):
    return (1000*Ao)/(997*x) 

In [4]:
x = 10
y = 8
k = x*y
k

80

In [5]:
Ai = 3

Ao = getAmountLPOut(Ai,x)

print('To remove {} of y, we need to extract {} LP'.format(Ai,Ao))

To remove 3 of y, we need to extract 29.91 LP


In [6]:
Ai = getAmountIn4LP(Ao,x)

print('We confirm that {} of y is required as input'.format(Ai))

We confirm that 3.0 of y is required as input


In [7]:
import numpy as np
xnew = x*(np.sqrt(1-kdel/k))
ynew = y*(np.sqrt(1-kdel/k))
xnew*ynew

NameError: name 'kdel' is not defined

In [ ]:
xdel = x*(1-np.sqrt(1-kdel/k)); xdel

In [ ]:
ydel = y*(1-np.sqrt(1-kdel/k)); ydel

In [ ]:
(x-xdel)*(y-ydel)

In [ ]:
(x-xdel)*(y-ydel)

In [ ]:
ydel1=1
(ydel1*1000 + (ydel1*1000 - ydel1*997))/1000

In [ ]:
ydel1 = (1000*kdel)/(997*x) 
(x)*(y-ydel1)

In [ ]:
ydel=1

In [ ]:
k - x*(y-997*ydel/1000)

In [ ]:
x*997*ydel/1000

In [ ]:
Ao = getAmountOut(ydel,x)
getAmountIn(Ao,x)

In [ ]:
def getAmountOut(ydel,R1):
    return R1*997*ydel/1000

In [ ]:
def getAmountIn(kdel,R1):
    return (1000*kdel)/(997*R1) 

In [ ]:
ydel1

In [ ]:
Tr = Ti/(Ti+To); Tr

#### Update reserveIn

In [ ]:
Ri = R - (Ao*Tr*1000/F)*(1 + Ao/(R-Ao)); Ri

#### Update reserveOut

In [ ]:
Ro = R - (Ao*(1-Tr)*1000/F)*(1 + Ao/(R-Ao)); Ro

In [ ]:
# (LRC eq.) Ri + Ro
Ri*Ro

#### Get amountIn

In [ ]:
Image("images/jupyter/random/delta_x.png", width = 600, height = 600)

In [ ]:
# (LRC eq.) Ai = -(1000*Ao*Tr*Ri)/(F*(Ao-Ro)); Ai
Ai = (1000*Ao*Ri)/(F*(Ro-Ao)); Ai

#### Get swap amount

In [ ]:
Image("images/jupyter/random/delta_y_math.png", width = 600, height = 600)

In [ ]:
Image("images/jupyter/random/delta_y_program.png", width = 600, height = 600)

In [ ]:
# (LRC eq.) S = (Ai*F*Ro)/(Ri*1000 + Ao*F); S
S = (Ai*F*Ro)/(Ri*1000 + Ai*F); S

In [ ]:
Ro

In [ ]:
# (LRC eq.) (R-Ro) + S